In [ ]:
# default_exp multi_model

In [ ]:
# hide
from nbdev.showdoc import *

# multi_model

> Class representing a list of `bcml_object` models, parameterized by a number

In [ ]:
# export
import numpy as np
import scipy.interpolate

In [ ]:
# export
class multi_model:
    """
    Represents a list of `bcml_object` models, parameterized by a number.
    """
    
    def __init__(self, models, index=None):
        self.models = models
        self.index = index if index is not None else [i for i in range(len(models))]
        
    def get_sigs(self, signals, background, tprs=None, fprs=None, preds=None, 
                 labels=None, sepbg=False, opt=False):
        """Returns a list of signal significances for the models in `self.models`"""
        tprs = tprs if tprs is not None else [None] * len(self.models) 
        fprs = fprs if fprs is not None else [None] * len(self.models) 
        return [
            model.significance(signal, background, tpr=tpr, fpr=fpr, sepbg=sepbg) 
            for model, signal, tpr, fpr in zip(self.models, signals, tprs, fprs)] if not opt else [
            model.best_threshold(signal, background, preds=preds, labels=labels, sepbg=sepbg)[1]
            for model, signal in zip(self.models, signals)
            ]
    
    def index2logsigF(self, signals, background, tprs=None, fprs=None, preds=None, labels=None, sepbg=False, opt=False):
        """Returns an interpolated function giving signal significances as a function of the `index` variable"""
        sigs = self.get_sigs(signals, background, tprs=tprs, fprs=fprs, preds=preds, labels=labels, sepbg=False, opt=opt)
        return scipy.interpolate.interp1d(self.index, np.log10(sigs), kind='cubic')
    
    def index2thresh_opt_improvementF(self, signal, background, tprs=None, fprs=None, sepbg=False, preds=None, 
                                      labels=None):
        """Returns an interpolated function giving the factor by which threshold optimization improves
        significance as a function of the `index` variable"""
        sigs = get_sigs(signal, background, tprs=tprs, fprs=fprs, sepbg=sepbg)
        opt_thresh_sigs = [
            model.best_threshold(signal, background, sepbg=sepbg, preds=preds, labels=labels) for model in self.models]
        improvement = [opt_thresh_sig/sig for opt_thresh_sig, sig in zip(sigs, opt_thresh_sigs)]
        return scipy.interpolate.interp1d(self.index, improvement, kind='cubic')
        
    def index2feature_importanceFs(self, features, num_features):
        """Returns a list of feature-function pairs, where each function interpolates its associated feature's
        importance as a function of the `index` variable"""
        feature_importancess = [model.sorted_feature_importance(features)[:num_features] for model in self.models]
        features = [[row[0] for row in feature_importances] for feature_importances in feature_importancess]
        # features present in the top "num_features" for each model in self.models
        present_features = list(set(features[0]).intersection(*features[1:]))
        importance_by_feature = [
            [feature, 
             [feature_importancess[i][feature_importancess[i].index(feature)][1] for i in range(len(self.models))]] 
            for feature in present_features]
        return [
             [feature, scipy.interpolate.interp1d(self.index, importances, kind='cubic')] 
             for feature, importances in importance_by_feature]

### Getting up and running

To create a `multi_model` object, you need to pass two arguments: a list of `bcml_model`s, and `index`, a list (with the same length) of real numbers parameterizing (or indexing) your models (often the associated new particle mass).

### Basic Functionality

In [ ]:
show_doc(multi_model.get_sigs)

<h4 id="multi_model.get_sigs" class="doc_header"><code>multi_model.get_sigs</code><a href="__main__.py#L11" class="source_link" style="float:right">[source]</a></h4>

> <code>multi_model.get_sigs</code>(**`signals`**, **`background`**, **`tprs`**=*`None`*, **`fprs`**=*`None`*, **`preds`**=*`None`*, **`labels`**=*`None`*, **`sepbg`**=*`False`*, **`opt`**=*`False`*)



In [ ]:
show_doc(multi_model.index2logsigF)

<h4 id="multi_model.index2logsigF" class="doc_header"><code>multi_model.index2logsigF</code><a href="__main__.py#L22" class="source_link" style="float:right">[source]</a></h4>

> <code>multi_model.index2logsigF</code>(**`signals`**, **`background`**, **`tprs`**=*`None`*, **`fprs`**=*`None`*, **`preds`**=*`None`*, **`labels`**=*`None`*, **`sepbg`**=*`False`*, **`opt`**=*`False`*)



In [ ]:
show_doc(multi_model.index2thresh_opt_improvementF)

<h4 id="multi_model.index2thresh_opt_improvementF" class="doc_header"><code>multi_model.index2thresh_opt_improvementF</code><a href="__main__.py#L26" class="source_link" style="float:right">[source]</a></h4>

> <code>multi_model.index2thresh_opt_improvementF</code>(**`signal`**, **`background`**, **`tprs`**=*`None`*, **`fprs`**=*`None`*, **`sepbg`**=*`False`*, **`preds`**=*`None`*, **`labels`**=*`None`*)



In [ ]:
show_doc(multi_model.index2feature_importanceFs)

<h4 id="multi_model.index2feature_importanceFs" class="doc_header"><code>multi_model.index2feature_importanceFs</code><a href="__main__.py#L34" class="source_link" style="float:right">[source]</a></h4>

> <code>multi_model.index2feature_importanceFs</code>(**`features`**, **`num_features`**)

